In [1]:
import pandas as pd
import numpy as np
import tqdm

In [2]:
recvae = pd.read_csv('/opt/ml/LSY/ensemble/RecVAE(top20)_epoch500_adam0.0005.csv')
lightfm = pd.read_csv('/opt/ml/LSY/ensemble/lightFM_50_500_ensemble_20_score.csv')

In [3]:
recvae = recvae.sort_values(by=['user','score'])
lightfm = lightfm.sort_values(by=['user','score'])

In [4]:
recvae = recvae.reset_index(drop=True)
lightfm = lightfm.reset_index(drop=True)

In [7]:
recvae

,user,item,score
0,11,3986,3.516525
1,11,4874,3.524286
2,11,55232,3.525939
3,11,40815,3.526372
4,11,8360,3.578262
...,...,...,...
627195,138493,2628,3.753575
627196,138493,1240,3.763737
627197,138493,589,3.787268
627198,138493,1270,3.851638


In [18]:
#hard voting
output_final = []

for i in range(31360):
    a = recvae['item'][20*i:20*(i+1)]
    b = lightfm['item'][20*i:20*(i+1)]
    b_sub_a = [x for x in b if x not in a]
    intersect =list(set(a)&set(b))
    if len(intersect)>10:
        intersect.sort(key = lambda x: np.where(b == x))
        output_final += intersect[-10:]
    elif len(intersect)<10:
        output_final += intersect
        output_final += b_sub_a[len(intersect)-10:]
    else:
        output_final += intersect

In [19]:
len(output_final)#313600

313600

In [22]:
submission = pd.read_csv("/opt/ml/input/data/eval/sample_submission.csv")
submission["item"] = output_final
submission.to_csv("./RecVAE_lightFM_ensemble_bytop20.csv", index=False)

In [2]:
recvae = pd.read_csv('/opt/ml/LSY/ensemble/RecVAE(top20)_epoch500_adam0.0005.csv')
lightfm = pd.read_csv('/opt/ml/LSY/ensemble/lightFM_50_500_ensemble_20_score.csv')
recvae = recvae.sort_values(by=['user','score'])
lightfm = lightfm.sort_values(by=['user','score'])

In [3]:
multivae = pd.read_csv('multivae.csv')
sasrec = pd.read_csv('sasrec.csv')

In [4]:
recvae = recvae.drop(['score'],axis=1)
lightfm = lightfm.drop(['score'],axis=1)
recvae['rank'] = 20 - recvae.index%20
lightfm['rank']=20 - lightfm.index%20

In [5]:
sasrec['rank']=sasrec.index%10+1
recvae['rank']=recvae.index%20+1

In [10]:
output = []
for i in range(31360):
    r = recvae[20*i:20*(i+1)]
    l = lightfm[20*i:20*(i+1)]
    s = sasrec[10*i:10*(i+1)]
    df = pd.merge(r,l,on = ['user','item'],how='outer')
    df = df.fillna(21)
    df = pd.merge(df, s, on=['user','item'],how = 'outer')
    df = df.fillna(11)
    df['sum rank'] = df['rank_x']+df['rank_y']+df['rank']
    df_sorted_by_values = df.sort_values(by=['sum rank','rank_x'] ,ascending=False)
    output.append(df_sorted_by_values['item'][0:10])

In [16]:
output = np.array(output)
output = output.reshape(1,-1)[0]

In [17]:
submission = pd.read_csv("/opt/ml/input/data/eval/sample_submission.csv")
submission["item"] = output
submission.to_csv("./sum_rank.csv", index=False)